<center><img src="img/slide-0.png"></center>

<center><img src="img/slide-1.png"></center>

## Required software for the tutorials

We will use [Conda](https://conda.io/projects/conda/en/latest/user-guide/getting-started.html)
to setup all required software for this tutorial. If you haven't setup
Conda yet, please do so first and then execute:

In [ ]:
!conda create -yqn tutorial -c conda-forge -c bioconda mmseqs2==12.113e3 plass== 4.687d7

In [ ]:
# in a shell you would run the following, however we want the binaries to be available in this notebook
# source activate tutorial
conda_path = !source activate tutorial && echo $CONDA_PREFIX:$PATH
%set_env PATH=$conda_path

# The Patient
<div style="display:flex; flex-direction:row;">
<div>
    
* 61-year-old man had bilateral headache, gait instability, lethargy, and confusion.
* Multiple tick bites in the preceding 2 weeks
* Possible rodent and bat exposures due to occupation
* Developed worsening confusion, weakness, and ataxia and was admitted
* Symptoms were diagnosed as Encephalitis; Causative agent - not known

* **Our task will be to identify the pathogenic root cause of the
disease.**
    
</div>
<div><img src="img/slide-29.png" style="max-height:500px;" alt="CT image of patients brain showing inflamation"></div>
</div>

<img src="img/slide-4.png" alt="This figure by Simner et al. shows a comparision of state of the art infections diagnosis methods compared to next generation sequence based diagnosis. Different type of pathogens require seperate tests, some lab test can take weeks to return a results while NGS can detect multiple types of pathogenes using the same protocol with in two days.">

<img src="img/slide-3.png" alt="In NGS based diagnosis we take a sample from the infectious site. For this patient the sample was taken from the Cerebrospinal fluid. CSF is a good sample side for brain infections. Directly sampling the infectious site is hard. We extract DNA and/or RNA from the sample by first breaking the cell walls. This is already a step that can cause the diagnosis to fail, some pathogens have stronger cell walls and require methods like bead beating to break them. Once we have extracted the DNA we sequence them using illumina. Now we end up with millions of short reads (DNA fragments), which are normally between 100-250 residues long. Our task today will be to taxonomically classify this reads by searching them against reference database. Once we have matched the reads to reference, we can visualize the taxonomical of a control sample and the infectious sample.">

This pathogen is usually confirmed by a screening antibody test,
followed by a plaque reduction neutralization test. However, this takes
5 weeks, which was too slow to affect the patient's care. As traditional
tests done in the first week of the patient's hospital stay did not
reveal any conclusive disease cause, the doctors were running out of
options. Therefore a novel metagenomic analysis was performed.

## Metagenomic Diagnosis

* Sequencing from brain biopsy performed on hospital day 8
* Returned 14 million short nucleotide sequences (reads)
* Authors released a set of 226,908 reads, human reads removed using Kraken (Wood et al 2014)

* **Why didn't the authors release the complete dataset?**

Most likely, because of **privacy** for the patient.

<center><img src="img/slide-39.png" alt="One popular method to label reads is Kraken. Kraken breaks the query sequence (read) into all overlapping k-mers (short words). Each k-mer is looked up in a pre-computed index table the respective taxonomical label. Once all k-mers all classified they are placed into a taxonomical tree. The taxon with the maximal path is assigned to the read. "></center>

## What are the limitations of nucleotide based classifications?

For many things we might **not have a closely related genomes** in the reference database.

### We can **use remote sequence homology to establish evolutionary relationships**.

<center><img src="img/slide-7.png" style="height:500px;" alt="Structural proteins are often conserved over a long period of time. Here you can see a ubiquitin from human and yeast. Both protein sequences branched off billions of year but the structure looks very similar. The conservation on the amino acid is not easy to see but some motives are still visible."></center>

## Metagenomic pathogen detection using MMseqs2

We will use MMseqs2 (Steinegger et al 2017) to find the cause of this patient's disease.
MMseqs2 translates the nucleotide reads to putative protein fragments, searches against a
protein reference database and assigns taxonomic labels based on the
found reference database hits.

<center><img src="img/slide-38.png" style="height:500px;" alt="We can just search all six possible open reading frames against the database. Six frames because there are three possible codon start position on the forward and additional tree on the reverse strand."></center>

<center><img src="img/slide-11.png" alt="MMseqs2 is a fast method to search reads six-frame translate again protein databases. It uses three filter stages (1) fast k-mer perfilter, (2) ungapped alignment and (3) an gapped alignment to improve its speed."></center>

<center><img src="img/slide-13.png" style="height:800px;" alt="Sensitivity of MMseqs2 can be adjusted based on the use case it can be very fast but less sensitive or as senstive than blast."></center>

<center><img src="img/slide-74.png" alt="You can see an sequence alignment with an e-value of 0.2. Do you trust this alignment?"></center>

<center><img src="img/slide-14.png"  style="height:800px;" alt="By aligning a profile against the sequence we can improve the E-value to 10^-17. PSI-BLAST is using this. Profiles are great to find remote homology connections." ></center>

<center><img src="img/slide-15.png" style="height:800px;" alt="MMSeqs2 can perform PSI-BLAST like searches but hundrets of times faster"></center>

Here you will learn the basic usage of MMseqs2 as well as expert tricks
to take advantage of the ability of chaining different MMseqs2 modules
to produce custom workflows.

The generic syntax for `mmseqs` and `plass` calls is always the
following:

    mmseqs <command> <db1> [<db2> ...] --flag --parameter 42

The help text of `mmseqs` shows, by default, only the most important
parameters and modules. To see a full list of parameters and modules use
the `-h` flag.

In [ ]:
!mmseqs

In [ ]:
!mmseqs -h

If you are using Bash as your shell, you can activate tab-auto-completion of commands and parameters:
`source $CONDA_PREFIX/util/bash-completion.sh`

# Downloading the reads
The sequencer returns paired-end reads where sequencing
starts in opposite directions from two close-by points to cover the
same genomic region. Some of these paired reads overlap enough to be
merged into a single read with FLASH (Magoc and Salzberg 2011).

To not spoil the mystery to early, we prepared a FASTA file containing the reads:

In [ ]:
!wget -nv -c http://wwwuser.gwdg.de/~compbiol/mmseqs2/tutorials/mystery_reads.fasta

# Reference database
We will use the
Swiss-Prot which is the manually curated, high-quality part of the
Uniprot (Uniprot Consortium 2014) protein reference database. Each protein in Uniprot has a taxonomic label. 

Usually you would use the `databases` workflow to automatically download and
prepare a wide range of reference databases (such the UniProt):

In [ ]:
!mmseqs databases

However, for this tutorial we want stable results and you are going to download a specific version we prepared for you:

In [ ]:
!wget -nv -c http://wwwuser.gwdg.de/~compbiol/mmseqs2/tutorials/uniprot_sprot_2018_03.fasta.gz
!wget -nv -c http://wwwuser.gwdg.de/~compbiol/mmseqs2/tutorials/uniprot_sprot_2018_03_mapping.tsv.gz
!gunzip uniprot_sprot_2018_03_mapping.tsv.gz

In [ ]:
!mmseqs createdb uniprot_sprot_2018_03.fasta.gz uniprot_sprot

In [ ]:
!mmseqs createtaxdb uniprot_sprot tmp --tax-mapping-file uniprot_sprot_2018_03_mapping.tsv

In [ ]:
!mmseqs createindex uniprot_sprot tmp

Through a similarity search we will transfer the annotation of the
reference protein to our unknown reads:

In [ ]:
!mmseqs createdb mystery_reads.fasta reads

In [ ]:
!mmseqs taxonomy reads uniprot_sprot lca_result tmp -s 2 

<center><img src="img/lca.png" alt="2bLCA is a protocol to assing taxon to sequences. It take uncertainty into account."></center>

MMseqs2 creates a result database in your working directory.
This database consists of files starting with `lca_result`. We
can convert this database into a human readable tab separated values
file (TSV):

In [ ]:
!mmseqs createtsv reads lca_result lca.tsv

In this file you see for every read a numeric taxonomic identifier, a
taxonomic rank and a taxonomic label. However, due to the large number
of reads, it is hard to gain any insights by skimming the file:

In [ ]:
!head -n 1000 lca.tsv

* **What are taxa with the highest read count in this dataset?**

In [ ]:
!cut -d $'\t' -f4 lca.tsv | sort | uniq -c | sort -rn | head

* **Why are there still 724 human reads?**

Kraken might have been not sensitive enough to annotate those.

* **Were all these bacterial really present in the brain sample?**

They might come from many different sources, such as lab-contamination, or reference-database contamination etc.

MMseqs2 offers a module to summarize the data into a single file `report.txt`:

In [ ]:
!mmseqs taxonomyreport uniprot_sprot lca_result report.txt 

In [ ]:
!cat report.txt

In [ ]:
from IPython.display import IFrame
IFrame('https://fbreitwieser.shinyapps.io/pavian/', width='100%', height=600)

### Visualizing taxonomic results

MMseqs2 can also generate an interactive visualization of the data using
Krona (Ondov et al 2011).
Adapt the previous call to generate a Krona report:

In [ ]:
!mmseqs taxonomyreport uniprot_sprot lca_result report.html --report-mode 1

This generates a `HTML` file that can be opened in a browser. This
offers an interactive circular visualization where you can click on each
label to zoom into different parts of the hierarchy.

In [ ]:
from IPython.display import IFrame
IFrame('report.html', width='100%', height=400)

## Using MMseqs2 Easy Workflows
All these steps can also be done in a single call. We call these easy workflows and they produce further useful output.

In [ ]:
!mmseqs easy-taxonomy mystery_reads.fasta uniprot_sprot easy_result tmp -s 2 

In [ ]:
!cat easy_result_tophit_report | grep powassan

<center><img src="img/uniprot_polyprotein.png" alt="Q04538 is a polyprotein. Meaning it encodes multiple "></center>

## Investigating the pathogen

We now want to take a closer look only at the reads of the pathogen. To
filter the result database, we will need the pathogen's numeric
taxonomic identifier. 

* **What is the taxon identifier of the powassan virus?**

In [ ]:
!grep -i powassan lca.tsv | head

Retrieve only the reads that belong to the powassan virus:

In [ ]:
!mmseqs filtertaxdb uniprot_sprot lca_result lca_only_pathogen --taxon-list 39008 

We need a list of all database keys of powassan to extract the respective reads:

In [ ]:
!mmseqs prefixid lca_only_pathogen lca_only_pathogen.tsv --tsv

With a few more commands we can convert our taxonomic labels back into a
FASTA file:

In [ ]:
!mmseqs createsubdb lca_only_pathogen.tsv reads reads_pathogen

In [ ]:
!mmseqs convert2fasta reads_pathogen reads_pathogen.fasta

* **How many reads of the pathogen are in this resulting FASTA file?**

In [ ]:
!grep -c "^>" reads_pathogen.fasta

<center><img src="img/slide-25.png" style="height:800px;" alt="Protein Level Assembly"></center>


<center><img src="img/slide-26.png" alt="Finding overlaps on the protein level instead using nucleotides residues reduces SNPs in the overlaps."></center>

<center><img src="img/slide-27.png" alt="PLASS is a protein assembler that can assemble multiple times more proteins from diverse enviroments compared to MEGAHIT/metaSPADes."></center>

<center><img src="img/slide-69.png" style="height:800px;" alt="Annika Seidel is currently developing an assembler, which can resolve full genomes from metagenomes."></center>

<center><img src="img/slide-73.png" alt="On ssRNA viral metatranscriptomes our assembler can assemble many time more sequences."></center>

We will use Plass-hybrid to find overlapping reads and generate contigs:

In [ ]:
!plass hybridassemble reads_pathogen.fasta pathogen_assembly.fasta tmp \
    --min-seq-id "nucl:0.95,aa:0.9" --min-contig-len 200 --min-length 20

Take a look at the generated FASTA file `pathogen_assembly.fasta`:


In [ ]:
!cat pathogen_assembly.fasta

* **How many sequences were assembled?**

In [ ]:
!grep -c "^>"  pathogen_assembly.fasta

### Clustering to find representative proteins (dereplication)

Plass uncovers a lot of variation in the reads and outputs many
similar proteins, which can be dereplicated using the sequence clustering module in MMseqs2.
<center><img src="img/slide-18.png" style="height:700px" alt="Sequence cluster is important to reduce redundancy or to build protein families"></center>

<center><img src="img/slide-19.png" alt="Clustering algorithm scale quadratically in runtime. But our algorithm Linclust scales linear in runtime"></center>

<center><img src="img/slide-20.png" style="height:800px;" alt="We pick a set of minimal hashing k-mers per sequence."></center>

<center><img src="img/slide-21.png" style="height:800px;" alt="We group them by soring."></center>

<center><img src="img/slide-22.png" style="height:800px;" alt="In each k-mer group we pick the longest sequences a cluster centroid."></center>

<center><img src="img/slide-23.png" style="height:800px;" alt="We now align the sequence members against the centroid."></center>

<center><img src="img/slide-24.png" style="height:800px;" alt="An assing the members to clusters"></center>

In [ ]:
!mmseqs easy-cluster pathogen_assembly.fasta assembly_clustered tmp --cov-mode 1 -c 0.8 

<center><img src="img/cov_modes.png" style="max-height:300px" alt="The coverage parameter is very important for Linclust." ></center>

You will see three files starting with `assembly_clustered` (`assembly_clustered_all_seqs.fasta`, `assembly_clustered_cluster.tsv`, `assembly_clustered_rep_seq.fasta`):


In [ ]:
!head -n 6 assembly_clustered_rep_seq.fasta assembly_clustered_cluster.tsv

* **How many sequences remain now?**

In [ ]:
!grep -c "^>" assembly_clustered_rep_seq.fasta

## Comparing assemblies with reference polyprotein

In [ ]:
!wget -O - https://www.uniprot.org/uniprot/Q04538.fasta \
    | mmseqs easy-search stdin assembly_clustered_rep_seq.fasta result.html tmp --format-mode 3

In [ ]:
from IPython.display import IFrame
IFrame('result.html', width='100%', height=800)

<center><img src="img/igv_view_powassan_reads.png" alt="Coverage across the genome is not uniform. Potential bias through experiment"></center>

### Annotating the contigs with the MMseqs2 Webserver

We will look for known protein domains to identify the proteins we
found. Instead of the MMseqs2 command line, we use the MMseqs2
webserver, which will visualize the results. Paste the content of the
FASTA file containing the representative sequences into the webserver
and make sure to select all three target databases (PFAM, PDB,
Uniclust): <https://search.mmseqs.com>

In [ ]:
from IPython.display import IFrame
IFrame('https://search.mmseqs.com/result/Rh6pCITfZXxJ7k_gsOy2nDKSOeW-TlRbKF5Tmw/0', width='100%', height=800)

## Aftermath

Despite being able to identify the causative agent. The pathogen is very
hard to treat. The patient had minimal neurological recovery and was
discharged to an acute care facility on hospital day 30. Seven months
after discharge, he was reportedly able to nod his head to questions and
slightly move his upper extremities and toes.

You can find the publication about this patient and dataset in
(Piantadosi et al. 2017).

## References

Piantadosi, Anne, Sanjat Kanjilal, Vijay Ganesh, Arjun Khanna, Emily P
Hyle, Jonathan Rosand, Tyler Bold, et al. 2017. "Rapid Detection of
Powassan Virus in a Patient With Encephalitis by Metagenomic
Sequencing." *Clinical Infectious Diseases* 66 (5): 789--92.

UniProt Consortium. 2014. "UniProt: A Hub for Protein Information."
*Nucleic Acids Research* 43 (D1): D204--D212.

Magoc, Tanja, and Steven L. Salzberg. 2011. "FLASH: Fast Length
Adjustment of Short Reads to Improve Genome Assemblies."
*Bioinformatics* 27 (21): 2957--63.

Ondov, Brian D, Nicholas H Bergman, and Adam M Phillippy. 2011.
"Interactive metagenomic visualization in a Web browser." *BMC
Bioinformatics* 12 (1): 385.

Wood, Derrick E, and Steven L Salzberg. 2014. "Kraken: ultrafast
metagenomic sequence classification using exact alignments." *Genome
Biology* 15 (3): R46.

<center><img src="img/slide-28.png"></center>

## Reset data to rerun tutorial

In [ ]:
!rm -rf reads* easy_result* lca* report.* pathogen_assembly.fasta assembly_clustered_* result.html tmp/

In [ ]:
!rm -rf uniport_sprot* mystery_reads.fasta

In [ ]:
    from IPython.core.display import display, HTML
    display(HTML(
        '<style>'
        '.rise-enabled #notebook-container { margin-left: auto !important; }'
        '/*div.output_area { max-height: 250px !important; }*/'
        '</style>'
    ))